## 局部加权线性回归
局部加权线性回归是普通线性回归的一个改进，普通的线性回归努力寻找一个使得全局代价函数最小的模型。这个模型对于整体来说是最好的，但对于局部点来说，可能不是最好的。比如图一是线性回归得到的模型，相对于图二来说，明显不够完美，图三虽然最完美但是存在过拟合风险。那么，如何用线性回归得到一个局部更准确（可能非线性模型拟合更好）的结果呢？这时，局部加权线性回归出现了。
![](局部加权线性回归.png)
局部加权线性回归是一种非参数算法，它不会得到一条适合于全局的函数模型，在每一次预测新样本时都会重新的确定参数，从而达到更好的预测效果。当数据规模比较大的时候计算量很大，学习效率很低。

其损失函数：$J(\theta)=\sum_{i=1}^mw^{(i)}(\hat{y}^{(i)}-y^{(i)})^2$，其中
$$w^{(i)}=\exp(-\frac{x^{(i)-x}}{2r^2})$$
其中x为待测点，r控制权值变化的速率，r越大，图像越瘦，离x越远权值下降越快。

In [1]:
from TV_examine.algorithm import locally_weighted_linearregression
from TV_examine.model_assessment import reg, divide_data
from TV_examine.data_preprocessing import preprocessing
import numpy as np
import matplotlib.pyplot as plt

## 读取数据
这里使用鲍鱼(一种水生物→_→)数据集，通过鲍鱼的特征来判断其年龄

In [2]:
fileName = "abalone.txt"
numFeat = len(open(fileName).readline().split('\t')) - 1
xArr = []; yArr = []
fr = open(fileName)
for line in fr.readlines():
    lineArr =[]
    curLine = line.strip().split('\t')
    for i in range(numFeat):
        lineArr.append(float(curLine[i]))
    xArr.append(lineArr)
    yArr.append(float(curLine[-1]))

## 观测数据
大致可以看出数据的详情，这里不做数据的预处理，只为测试局部加权线性回归算法

In [10]:
X = np.array(xArr)
y = np.array(yArr)
X, y ,X.shape

(array([[ 1.    ,  0.455 ,  0.365 , ...,  0.2245,  0.101 ,  0.15  ],
        [ 1.    ,  0.35  ,  0.265 , ...,  0.0995,  0.0485,  0.07  ],
        [-1.    ,  0.53  ,  0.42  , ...,  0.2565,  0.1415,  0.21  ],
        ...,
        [ 1.    ,  0.6   ,  0.475 , ...,  0.5255,  0.2875,  0.308 ],
        [-1.    ,  0.625 ,  0.485 , ...,  0.531 ,  0.261 ,  0.296 ],
        [ 1.    ,  0.71  ,  0.555 , ...,  0.9455,  0.3765,  0.495 ]]),
 array([15.,  7.,  9., ...,  9., 10., 12.]),
 (4177, 8))

## 数据处理及训练
- 这里使用留出法对数据进行划分。留出法：将数据集划分为两个互斥的部分，一部分做训练集一部分做测试集。以下测试集与训练集值比值为0.2.
- 输入不同的k值，观测其拟合效果

In [4]:
X_train, X_test, y_train, y_test = divide_data.train_test_split(X, y, 0.2)
lwlr = locally_weighted_linearregression.LWLR()
lwlr.fit(X_train, y_train)

In [6]:
lwlr.X.shape
y_pr2 = lwlr.predict(X_test, 1)
y_pr3 = lwlr.predict(X_test, 10)

## 训练评估
可以看出k为1和10训练的区别不是很大，但是仍然可以分辨出k为1时模型稍优于k为10时的模型，如果把k再提高点模型或许将会过拟合。

In [7]:
ass2 = reg.reg_assessment(y_pr2, y_test)
ass3 = reg.reg_assessment(y_pr3, y_test)

In [8]:
ass2.assessment()

均方误差（MSE）： 4.911599334989377
平均绝对误差（MAE）： 1.5610959595302942
均方根误差（RMSE）： 2.216212836121427
平均绝对百分比误差（MAPE）： 0.15770710467267116
R平方： 0.999429708347846


In [9]:
ass3.assessment()

均方误差（MSE）： 5.02338785594275
平均绝对误差（MAE）： 1.5958844049282759
均方根误差（RMSE）： 2.2412915597803758
平均绝对百分比误差（MAPE）： 0.16338316991923682
R平方： 0.9994167284494547
